## Question 1: **Answer**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(input_data):
    try:
        data = pd.read_csv(input_data)
        engine = sqlite3.connect('library_records.db')
        data.to_sql('library', con=engine, index = False)
    except Exception as e:
        print(f"Error: {e}")

data = "/content/library_data.csv"
add_data_to_database(data)

In [3]:
# Loading SQL extension
%load_ext sql

In [4]:
# Connecting to the database
%sql sqlite:///library_records.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM library
LIMIT 5;

 * sqlite:///library_records.db
Done.


library_card_number,borrower_name,age_group,book_title,author,genre,due_date,borrowed_date
1000,Alice Green,Child,The Very Hungry Caterpillar,Eric Carle,Picture Book,2020-02-15,2020-01-20
1001,Bob Brown,Teen,Percy Jackson & the Olympians: The Lightning Thief,Rick Riordan,Fantasy,2020-02-25,2020-02-10
1002,Charlie White,Teen,To Kill a Mockingbird,Harper Lee,Classic,2020-03-10,2020-02-05
1003,David Miller,Adult,The Martian,Andy Weir,Science Fiction,2020-02-20,2020-02-02
1004,Emily Jones,Adult,Pride and Prejudice,Jane Austen,Classic,2020-02-18,2020-01-30


## Question 2: **Answer**

In [7]:
%%sql
SELECT
    age_group,
    book_title
FROM library
WHERE book_title LIKE "The Hitchhiker's Guide to the Galaxy";

 * sqlite:///library_records.db
Done.


age_group,book_title
Adult,The Hitchhiker's Guide to the Galaxy


## Question 3: **Answer**

In [8]:
%%sql
--CTE identifies top 3 genres based on the number of borrows.
WITH book_borrows AS (
    SELECT
        genre,
        COUNT(*) AS total_borrows
    FROM library
    GROUP BY genre
    ORDER BY total_borrows DESC
    LIMIT 3)

-- Calculating total borrows
SELECT
    SUM(total_borrows) AS number_of_top_3_borrows
FROM book_borrows;

 * sqlite:///library_records.db
Done.


number_of_top_3_borrows
13


## Question 4: **Answer**

In [9]:
%%sql
SELECT
    borrower_name,
    book_title,
    borrowed_date,
    due_date
FROM library
WHERE due_date <= '2020-02-20';

 * sqlite:///library_records.db
Done.


borrower_name,book_title,borrowed_date,due_date
Alice Green,The Very Hungry Caterpillar,2020-01-20,2020-02-15
David Miller,The Martian,2020-02-02,2020-02-20
Emily Jones,Pride and Prejudice,2020-01-30,2020-02-18
James Green,A Brief History of Time,2020-01-23,2020-02-16
Katie Green,Divergent,2020-01-31,2020-02-19
Peter Jones,The Count of Monte Cristo,2020-01-29,2020-02-17


## Question 5: **Answer**

In [10]:
%%sql
SELECT
    age_group,
    COUNT(*) AS borrow_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM library), 2)
    AS percentage_of_total_borrowing
FROM library
GROUP BY age_group;

 * sqlite:///library_records.db
Done.


age_group,borrow_count,percentage_of_total_borrowing
Adult,13,59.09
Child,4,18.18
Teen,5,22.73


## Question 6: **Answer**

In [11]:
%%sql
SELECT
     genre,
     ROUND(AVG(julianday(due_date) - julianday(borrowed_date)), 2) AS
     avg_borrowing_time
FROM library
GROUP BY genre
ORDER BY avg_borrowing_time DESC;

 * sqlite:///library_records.db
Done.


genre,avg_borrowing_time
Biography,37.0
Historical Fiction,34.0
Fantasy,26.0
Classic,24.6
Science,24.0
Picture Book,22.0
Science Fiction,21.67
Dystopian,20.25
Fiction,14.0
